In [3]:
import pandas as pd
from pathlib import Path
data_dir = Path('semantic_search/data/abo/')
csv = data_dir / 'images' /'metadata'/'images.csv.gz'
listings_dir = data_dir/ 'listings'/'metadata'

In [2]:
df = pd.read_csv(csv)
df.head()

,image_id,height,width,path
0,010-mllS7JL,106,106,14/14fe8812.jpg
1,01dkn0Gyx0L,122,122,da/daab0cad.jpg
2,01sUPg0387L,111,111,d2/d2daaae9.jpg
3,1168jc-5r1L,186,186,3a/3a4e88e6.jpg
4,11RUV5Fs65L,30,500,d9/d91ab9cf.jpg


In [5]:
listing_jsons = [f for f in listings_dir.iterdir()]

In [6]:
df_list0 = pd.read_json(listing_jsons[0], lines=True)
df_list0.head()

,brand,bullet_point,color,fabric_type,item_id,item_name,model_name,model_number,product_type,style,...,item_keywords,material,spin_id,3dmodel_id,color_code,model_year,pattern,product_description,finish_type,item_shape
0,"[{'language_tag': 'de_DE', 'value': 'Amazon Es...","[{'language_tag': 'de_DE', 'value': 'Fällt gro...","[{'language_tag': 'de_DE', 'value': 'Mehrfarbi...","[{'language_tag': 'en_GB', 'value': '100% Cott...",B07HL25ZQM,"[{'language_tag': 'en_GB', 'value': 'Amazon Es...","[{'language_tag': 'en_GB', 'value': '6-Pack Bi...",[{'value': 'P_AE3131_M6'}],[{'value': 'BABY_PRODUCT'}],"[{'language_tag': 'de_DE', 'value': '6-Pack Bi...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[{'language_tag': 'en_GB', 'value': 'AmazonBas...","[{'language_tag': 'en_GB', 'value': 'Large dry...",NaN,NaN,B0825D4F6R,"[{'language_tag': 'en_GB', 'value': 'AmazonBas...",NaN,[{'value': 'AMAZ2001'}],[{'value': 'HOME'}],"[{'language_tag': 'en_GB', 'value': 'Deluxe'}]",...,"[{'language_tag': 'en_GB', 'value': 'tower lau...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[{'language_tag': 'en_IN', 'value': 'Amazon Br...","[{'language_tag': 'en_IN', 'value': '3D Printe...","[{'language_tag': 'en_IN', 'standardized_value...",NaN,B07TF1FCFD,"[{'language_tag': 'en_IN', 'value': 'Amazon Br...","[{'language_tag': 'en_IN', 'value': 'Samsung G...",[{'value': 'gz8587-SL40668'}],[{'value': 'CELLULAR_PHONE_CASE'}],NaN,...,"[{'language_tag': 'en_IN', 'value': 'mobile co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[{'language_tag': 'en_IN', 'value': 'Amazon Br...","[{'language_tag': 'en_IN', 'value': 'Snug fit ...","[{'language_tag': 'en_IN', 'standardized_value...",NaN,B08569SRJD,"[{'language_tag': 'en_IN', 'value': 'Amazon Br...","[{'language_tag': 'en_IN', 'value': 'Nokia 7.2'}]",[{'value': 'UV10845-SL40357'}],[{'value': 'CELLULAR_PHONE_CASE'}],NaN,...,"[{'language_tag': 'en_IN', 'value': 'Back Cove...","[{'language_tag': 'en_IN', 'value': 'Silicon'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[{'language_tag': 'en_US', 'value': 'Stone & B...","[{'language_tag': 'en_US', 'value': 'With mode...","[{'language_tag': 'en_US', 'value': 'Dark Grey'}]",NaN,B07B4G5RBN,"[{'language_tag': 'zh_CN', 'value': 'Stone & B...",NaN,[{'value': 'UPH10095B'}],[{'value': 'CHAIR'}],NaN,...,"[{'language_tag': 'en_US', 'value': 'living-ro...","[{'language_tag': 'zh_CN', 'value': '灰石色'}, {'...",485925ed,B07B4G5RBN,[#918F8C],NaN,NaN,NaN,NaN,NaN
